## Interacting with Web APIs

In our first class, we examined how to use `curl` to issue requests against web services. We will now see how to achieve the same in Python:

In [1]:
# We first import the requests library
import requests
url = 'http://freegeoip.net/json/'
resp = requests.get(url)

In [2]:
# The resp object encapsulates the "response" of the server
# Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong
resp

<Response [200]>

In [3]:
if (resp.status_code == 200):
    print "Everything was ok:", resp.status_code
else:
    print "There was a problem:", resp.status_code

Everything was ok: 200


In [11]:
# Let's see the content of the response
# As you can see, it contain the JSON response
resp.text

u'{"ip":"54.174.159.22","country_code":"US","country_name":"United States","region_code":"VA","region_name":"Virginia","city":"Ashburn","zip_code":"20147","time_zone":"America/New_York","latitude":39.0335,"longitude":-77.4838,"metro_code":511}\n'

In [12]:
# We want to transform the JSON file into a Python dictionary object
# For that we will use the json library
import json

# The loads (LOADS = LOAD from String) function reads a string that represents a JSON file
data = json.loads(resp.text)

In [13]:
# Now data is a Python dictionary
# The u'....' characters mean that the string is represented in Unicode
data

{u'city': u'Ashburn',
 u'country_code': u'US',
 u'country_name': u'United States',
 u'ip': u'54.174.159.22',
 u'latitude': 39.0335,
 u'longitude': -77.4838,
 u'metro_code': 511,
 u'region_code': u'VA',
 u'region_name': u'Virginia',
 u'time_zone': u'America/New_York',
 u'zip_code': u'20147'}

In [7]:
# And we can access the fields of the JSON as we normally access Python dictionary entries
print "Lon:", data["longitude"], "Lat:", data["latitude"]

Lon: -77.4838 Lat: 39.0335


### Exercise

Read the location of your computer using the GeoIP API, and then use the OpenWeatherMap to query the API and fetch the temperature ((Documentation)[http://openweathermap.org/current#geo]). For this exercise, you will need to:
* Create an account with OpenweatherMap and get an API key
* Study the documentation of the API on OpenWeatherMap
* Learn to read variables from a Web API (freegeoip) and use them as input in another (openweathermap)

In [14]:
import requests
import json
url = 'http://freegeoip.net/json/'
resp = requests.get(url)
data = json.loads(resp.text)
lon = data["longitude"]
lat = data["latitude"]

#your code here


In [24]:
url2 = "http://api.openweathermap.org/data/2.5/weather?" + \
     "&lat=" + str(lat) + \
     "&lon=" + str(lon) + \
     "&appid=44db6a862fba0b067b1930da0d769e98" + \
     "&units=imperial"

resp = requests.get(url2)
data = json.loads(resp.text)
data
print data["main"]["temp"]

40.24


### Solution for Exercise

In [ ]:
import requests
import json
url = 'http://freegeoip.net/json/'
resp = requests.get(url)
data = json.loads(resp.text)
lon = data["longitude"]
lat = data["latitude"]

# http://api.openweathermap.org/data/2.5/weather?q=New%20York,NY,USA

url2 = 'http://api.openweathermap.org/data/2.5/weather?' + \
    '&lat=' + str(lat) + \
    '&lon=' + str(lon) + \
    '&appid=ffb7b9808e07c9135bdcc7d1e867253d' + \
    '&units=imperial' + \
    '&mode=json'
print url2
resp = requests.get(url2)
data = json.loads(resp.text)
data

## Beyond the basics: Parameters and Headers

The first call that we submitted was very simple. We just fetched a URL, which was represented as a string. However, calling web services by manually concatenating strings to create URLs is not a good practice. It is always better to use a dictionary to pass the parameters to the API call; furthermore, we will often need to pass a set of _headers_ to the API call (as in the case of Mashape).

Let's play a little bit with the FacePlusPlus API, with which we toyed around in our first session:

![Image from NY Times](http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg)

In [ ]:
curl --get 
'https://faceplusplus-faceplusplus.p.mashape.com/detection/detect'
'?attribute=glass%2Cpose%2Cgender%2Cage%2Crace%2Csmiling
'&url=http%3A%2F%2Fwww.faceplusplus.com%2Fwp-content%2Fthemes%2Ffaceplusplus%2Fassets%2Fimg%2Fdemo%2F1.jpg' \
  -H 'X-Mashape-Key: zG3wec50exmshxNoF1NMHNRH37GYp1d7oW8jsnWwIMTeMmALxg' \
  -H 'Accept: application/json'

In [28]:
import requests
import json

facepp_url = "https://faceplusplus-faceplusplus.p.mashape.com/detection/detect"
img_url = "http://previews.123rf.com/images/andresr/andresr1206/andresr120600543/14095175-Happy-group-of-students-smiling-isolated-over-a-white-background--Stock-Photo.jpg"

headers = {
  "X-Mashape-Key": "zG3wec50exmshxNoF1NMHNRH37GYp1d7oW8jsnWwIMTeMmALxg",
  "Accept": "application/json"
}
parameters = {
    'attributes': 'glass,pose,gender,age,race,smiling',
    'url': img_url
}

resp = requests.get(facepp_url, params=parameters, headers=headers, verify=False)
data = json.loads(resp.text)

data

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'face': [{u'attribute': {u'age': {u'range': 9, u'value': 29},
    u'gender': {u'confidence': 99.9999, u'value': u'Female'},
    u'race': {u'confidence': 77.1456, u'value': u'White'},
    u'smiling': {u'value': 97.2266}},
   u'face_id': u'294ed7a68890e233fca09946ccf3bfe0',
   u'position': {u'center': {u'x': 81.0, u'y': 28.571429},
    u'eye_left': {u'x': 78.280833, u'y': 25.719532},
    u'eye_right': {u'x': 82.6735, u'y': 24.860312},
    u'height': 15.064935,
    u'mouth_left': {u'x': 79.124833, u'y': 32.843636},
    u'mouth_right': {u'x': 83.489667, u'y': 31.681039},
    u'nose': {u'x': 79.2865, u'y': 29.334286},
    u'width': 9.666667},
   u'tag': u''},
  {u'attribute': {u'age': {u'range': 6, u'value': 18},
    u'gender': {u'confidence': 99.9957, u'value': u'Male'},
    u'race': {u'confidence': 99.5775, u'value': u'White'},
    u'smiling': {u'value': 97.3315}},
   u'face_id': u'95d44bc24ad4b20e7c849bccc0282f2b',
   u'position': {u'center': {u'x': 67.083333, u'y': 23.506494},
    u'e

In [30]:
# We can also pretty print the dictionary object
# print(json.dumps(data, indent=2))

In [31]:
# And here is a more generic way to pretty print Python data structures, which works for many objects
# import pprint
# pprint.pprint(data)

In [33]:
len(data["face"])

5

### Interacting with Alchemy API; POST vs GET

Sometimes, the parameters that we need to pass to the API are too long (e.g., analyzing a piece of long text for sentiment). In such cases, we need to use the "POST" options as opposed to the "GET" function (although admittedly, many APIs will try to be forgiving).

In [34]:
import requests
import json

url = "http://access.alchemyapi.com/calls/text/TextGetTextSentiment"

# You can register and get your own key
api_key = '3d0b6858f7ef32fdf27ad402f4a9c270c9685d84'

text = '''
If things go as promised, Trump won’t be there Thursday when Fox hosts the final Republican debate before Monday’s Iowa presidential caucuses. He says he’s backing out because of a taunting statement from Fox, though his detractors accuse him of dodging a last showdown with his chief rival, Sen. Ted Cruz (Tex.). Instead, Trump has made plans to materialize elsewhere in Iowa, hosting a benefit for wounded veterans — counter-programming on a ­Trumpian scale of swagger.

His threatened absence from the debate stage is a demonstration of Trump’s perception of his own self-worth, his verifiable status as a ratings-generating gargantuan whose screen persona can translate into millions of advertising dollars. In a sense, it’s an act of subversion by a candidate who has broken all the normal rules of modern campaigns. But it’s also a manifestation of Trump’s philosophy about getting what he wants when he wants it.
'''

headers = {
  "Accept": "application/json"
}

parameters = {
    'outputMode': 'json',
    'apikey' : api_key,
    'text': text,
}

resp = requests.post(url, params=parameters, headers=headers)
data = json.loads(resp.text)

data

{u'docSentiment': {u'mixed': u'1',
  u'score': u'0.026919',
  u'type': u'positive'},
 u'language': u'english',
 u'status': u'OK',
 u'totalTransactions': u'1',
 u'usage': u'By accessing AlchemyAPI or using information generated by AlchemyAPI, you are agreeing to be bound by the AlchemyAPI Terms of Use: http://www.alchemyapi.com/company/terms.html'}

And here is a different API call that extracts entities from the text, and also the sentiment for each of these entities. Furthermore, each entity is matched into a "normalized" entry in the Knowledge Graph.

In [35]:
import requests
import json

url = "http://gateway-a.watsonplatform.net/calls/url/URLGetRankedNamedEntities"


# You can register and get your own key
api_key = '3d0b6858f7ef32fdf27ad402f4a9c270c9685d84'


text_url = 'https://www.washingtonpost.com/politics/question-whats-clintons-message-answer-all-of-the-above/2016/02/06/5df28fcc-cc4a-11e5-88ff-e2d1b4289c2f_story.html'
headers = {
  "Accept": "application/json"
}

parameters = {
    'outputMode': 'json',
    'apikey' : api_key,
     'sentiment' :1,
    'knowledgeGraph': 1,
    'url': text_url
}

resp = requests.post(url, params=parameters, headers=headers)
data = json.loads(resp.text)

print data["entities"]

{u'entities': [{u'count': u'16',
   u'disambiguated': {u'dbpedia': u'http://dbpedia.org/resource/Hillary_Rodham_Clinton',
    u'freebase': u'http://rdf.freebase.com/ns/m.0d06m5',
    u'name': u'Hillary Rodham Clinton',
    u'opencyc': u'http://sw.opencyc.org/concept/Mx4rvV7SqpwpEbGdrcN5Y29ycA',
    u'subType': [u'Politician',
     u'Appointee',
     u'AwardWinner',
     u'BoardMember',
     u'Celebrity',
     u'HallOfFameInductee',
     u'OperaCharacter',
     u'Senator',
     u'U.S.Congressperson',
     u'TVActor'],
    u'website': u'http://www.state.gov/secretary/index.htm',
    u'yago': u'http://yago-knowledge.org/resource/Hillary_Rodham_Clinton'},
   u'knowledgeGraph': {u'typeHierarchy': u'/people/politicians/democrats/hillary clinton'},
   u'relevance': u'0.889601',
   u'sentiment': {u'mixed': u'1',
    u'score': u'-0.245827',
    u'type': u'negative'},
   u'text': u'Hillary Clinton',
   u'type': u'Person'},
  {u'count': u'12',
   u'disambiguated': {u'dbpedia': u'http://dbpedia.or

In [38]:
for person in data["entities"]:
    print person["text"], person["relevance"]

Hillary Clinton 0.889601
Sen. Bernie Sanders 0.541743
Clinton 0.364824
Bill Clinton 0.294234
President Obama 0.235373
Congress 0.234005
Manchester 0.211838
Washington Post 0.203343
N.H. 0.19979
Nick Pangaro 0.196862
Derry 0.184953
Chuck Todd 0.182349
presidential debate 0.175534
Iowa 0.174967
CONCORD 0.174002
clean energy 0.16506
Rep. Tim Roemer 0.164432
President 0.163938
Joel Benenson 0.163235
Vermont 0.159861
Brian Fallon 0.156816
MSNBC 0.153619
foreign policy 0.151835
Mich. 0.147012
ISIS 0.144533
Abby Phillip 0.143875
New Hampshire primary 0.139845
New Hampshire 0.139068
consultant 0.136349
Flint 0.131449
Boys and Girls Club 0.131275
Durham 0.129793
Russia 0.129725
Medicare 0.128941
Mika Brzyznski 0.12633
insurance system 0.125675
Iran 0.117628
tumulty@washpost.com 0.117628
gearan@washpost.com 0.117628


In [40]:
import pandas as pd
from pandas.io.json import json_normalize
df = json_normalize(data["entities"])
df

,count,disambiguated.census,disambiguated.crunchbase,disambiguated.dbpedia,disambiguated.freebase,disambiguated.geo,disambiguated.geonames,disambiguated.musicBrainz,disambiguated.name,disambiguated.opencyc,disambiguated.subType,disambiguated.website,disambiguated.yago,knowledgeGraph.typeHierarchy,relevance,sentiment.mixed,sentiment.score,sentiment.type,text,type
0,16,NaN,NaN,http://dbpedia.org/resource/Hillary_Rodham_Cli...,http://rdf.freebase.com/ns/m.0d06m5,NaN,NaN,NaN,Hillary Rodham Clinton,http://sw.opencyc.org/concept/Mx4rvV7SqpwpEbGd...,"[Politician, Appointee, AwardWinner, BoardMemb...",http://www.state.gov/secretary/index.htm,http://yago-knowledge.org/resource/Hillary_Rod...,/people/politicians/democrats/hillary clinton,0.889601,1,-0.245827,negative,Hillary Clinton,Person
1,12,NaN,NaN,http://dbpedia.org/resource/Bernie_Sanders,http://rdf.freebase.com/ns/m.01_gbv,NaN,NaN,NaN,Bernie Sanders,NaN,"[Politician, Senator, U.S.Congressperson, Film...",NaN,http://yago-knowledge.org/resource/Bernie_Sanders,/senators/bernie sanders/sen. bernie sanders,0.541743,1,-0.190996,negative,Sen. Bernie Sanders,Person
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/people/clinton,0.364824,NaN,-0.524108,negative,Clinton,Person
3,1,NaN,NaN,http://dbpedia.org/resource/Bill_Clinton,http://rdf.freebase.com/ns/m.0157m,NaN,NaN,http://zitgist.com/music/artist/a11bd200-7f0b-...,Bill Clinton,http://sw.opencyc.org/concept/Mx4rwQBp5JwpEbGd...,"[MusicalArtist, Politician, Appointer, AwardWi...",http://www.clintonlibrary.gov/,http://yago-knowledge.org/resource/Bill_Clinton,/people/politicians/presidents/bill clinton,0.294234,NaN,NaN,neutral,Bill Clinton,Person
4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/people/politicians/democrats/president obama,0.235373,NaN,-0.321666,negative,President Obama,Person
5,3,NaN,NaN,http://dbpedia.org/resource/United_States_Cong...,http://rdf.freebase.com/ns/m.07t31,NaN,NaN,NaN,United States Congress,http://sw.opencyc.org/concept/Mx4rwP1W2JwpEbGd...,"[Dedicator, GovernmentalBody, Legislature, Nat...",http://www.house.gov/,http://yago-knowledge.org/resource/United_Stat...,/activities/events/parties/congress,0.234005,NaN,-0.306341,negative,Congress,Organization
6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/places/cities/manchester,0.211838,1,-0.0283284,negative,Manchester,City
7,2,NaN,NaN,http://dbpedia.org/resource/The_Washington_Post,http://rdf.freebase.com/ns/m.0px38,NaN,NaN,NaN,The Washington Post,NaN,[Newspaper],http://www.washingtonpost.com,http://yago-knowledge.org/resource/The_Washing...,/organizations/media/publications/washington post,0.203343,NaN,0.277664,positive,Washington Post,PrintMedia
8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/towns/n.h.,0.19979,NaN,NaN,neutral,N.H.,StateOrCounty
9,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/problems/defects/nicks/nick pangaro,0.196862,NaN,NaN,neutral,Nick Pangaro,Person
